In [ ]:
import os
import numpy
import pandas
idx=pandas.IndexSlice
import time
import random
import matplotlib
#%matplotlib notebook
import matplotlib.pyplot as plt
import scipy.stats
#from pandas.plotting import autocorrelation_plot
import matplotlib.offsetbox as offsetbox
from matplotlib.ticker import StrMethodFormatter
import sklearn.linear_model


In [ ]:
#for some reason, this needs to be in a separate cell
params={
    "font.size":15,
    "lines.linewidth":5
}
plt.rcParams.update(params)

In [ ]:
def getfile(location_pair,**kwargs): #tries to get local version and then defaults to google drive version
    (loc,gdrive)=location_pair
    try:
        out=pandas.read_csv(loc,**kwargs)
    except FileNotFoundError:
        print("local file not found; accessing Google Drive")
        loc = 'https://drive.google.com/uc?export=download&id='+gdrive.split('/')[-2]
        out=pandas.read_csv(loc,**kwargs)
    return out

In [ ]:
fname=("uiuc-gpa-dataset.csv","https://drive.google.com/file/d/17iMVJ64MWmxCTf-oP779c3vrA3UGFCOI/view?usp=sharing")

In [ ]:
data_raw = getfile(fname)
data_raw.head()

local file not found; accessing Google Drive


,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,B-,C+,C,C-,D+,D,D-,F,W,Primary Instructor
0,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,2,14,2,...,3,0,1,0,0,0,0,0,0,"Zheng, Reanne"
1,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,0,15,0,...,1,0,2,0,0,0,0,1,0,"Atienza, Paul Michael L"
2,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,OD,7,4,1,...,0,2,3,0,0,1,0,1,0,"Wang, Yu"
3,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,1,18,0,...,1,0,0,0,0,0,0,0,0,"Zheng, Reanne"
4,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,0,16,1,...,1,0,2,0,0,0,0,0,0,"Atienza, Paul Michael L"


In [ ]:
#data_raw.to_pickle("./working_dataset.pkl")

In [ ]:
data_raw['Subject'].value_counts()

ECON    3498
ACCY    2543
MATH    2482
BADM    2446
MCB     2107
        ... 
GRKM       2
CB         2
JS         2
HNDI       1
DTX        1
Name: Subject, Length: 168, dtype: int64

In [ ]:
debugging_df = data_raw.groupby('Subject').filter(lambda x: len(x)>10).sample(n=500, random_state=42)
debugging_df.head()

,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,B-,C+,C,C-,D+,D,D-,F,W,Primary Instructor
28714,2016,Spring,2016-sp,ECON,203,Economic Statistics II,LAB,0,5,2,...,1,6,6,1,2,2,6,0,0,"Stevens, Alexander N"
34996,2015,Summer,2015-su,ANSC,331,Biology of Reproduction,ONL,6,6,4,...,0,0,2,0,0,0,0,0,0,"Nowak, Romana A"
9827,2020,Winter,2020-wi,FSHN,120,Contemporary Nutrition,ONL,0,58,0,...,0,0,6,0,0,3,0,0,1,"Burkhalter, Toni M"
53466,2011,Summer,2011-su,ACCY,512,Risk Measurement/Reporting II,LCD,2,25,6,...,0,0,0,0,0,0,0,0,0,"Schwartz, Rachel"
57642,2010,Fall,2010-fa,HIST,439,The Ottoman Empire,LCD,1,11,3,...,2,1,0,0,0,0,0,3,1,"Todorova, Maria N"


In [ ]:
debugging_df['Subject'].value_counts()

ECON    30
BADM    24
CS      22
MATH    22
PSYC    19
        ..
REL      1
ACES     1
ENSU     1
ATMS     1
HRD      1
Name: Subject, Length: 100, dtype: int64

In [ ]:
# debugging_df.to_pickle("./debugging_dataset.pkl")

In [ ]:
def getpickle(location_pair,**kwargs): #tries to get local version and then defaults to google drive version
    (loc,gdrive)=location_pair
    try:
        out=pandas.read_pickle(loc,**kwargs)
    except FileNotFoundError:
        print("local file not found; accessing Google Drive")
        loc = 'https://drive.google.com/uc?export=download&id='+gdrive.split('/')[-2]
        out=pandas.read_pickle(loc,**kwargs)
    return out

In [ ]:
fname=("working_dataset.pkl", "https://drive.google.com/file/d/1ihNXt1tef8zR7CqvAtooysEC7b5dyYkZ/view?usp=sharing")
data = getpickle(fname) 
data.head()

local file not found; accessing Google Drive


,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,B-,C+,C,C-,D+,D,D-,F,W,Primary Instructor
0,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,2,14,2,...,3,0,1,0,0,0,0,0,0,"Zheng, Reanne"
1,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,0,15,0,...,1,0,2,0,0,0,0,1,0,"Atienza, Paul Michael L"
2,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,OD,7,4,1,...,0,2,3,0,0,1,0,1,0,"Wang, Yu"
3,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,1,18,0,...,1,0,0,0,0,0,0,0,0,"Zheng, Reanne"
4,2021,Fall,2021-fa,AAS,100,Intro Asian American Studies,DIS,0,16,1,...,1,0,2,0,0,0,0,0,0,"Atienza, Paul Michael L"


In [ ]:
print(data.shape)

(61557, 22)


In [ ]:
data.columns

Index(['Year', 'Term', 'YearTerm', 'Subject', 'Number', 'Course Title',
       'Sched Type', 'A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+',
       'D', 'D-', 'F', 'W', 'Primary Instructor'],
      dtype='object')

In [ ]:
data['Year'].value_counts()

2011    7970
2010    5423
2015    5218
2017    5093
2021    5092
2013    5045
2019    5041
2018    5029
2016    4969
2014    4957
2020    4855
2012    2865
Name: Year, dtype: int64